In [ ]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image, ImageOps
from tqdm import tqdm
from os.path import join
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
from torchvision import transforms
device='cuda:0'

In [ ]:
train_df = pd.read_csv('/data/wikiart/wikiart_Painting100k/MultitaskPainting100k_Dataset_groundtruth/groundtruth_multiloss_train_header.csv')
valid_df = pd.read_csv('/data/wikiart/wikiart_Painting100k/MultitaskPainting100k_Dataset_groundtruth/groundtruth_multiloss_test_header.csv')
print(len(train_df), len(valid_df))

train_df['img_path'] = train_df.apply(lambda x: join('/data/wikiart/wikiart_Painting100k/images_256minside',x.filename),1)
valid_df['img_path'] = valid_df.apply(lambda x: join('/data/wikiart/wikiart_Painting100k/images_256minside',x.filename),1)

class_dict_genre = {}
for i, genre in enumerate(np.sort(train_df.genre.unique())):
    train_df.loc[train_df.genre==genre, 'class_genre'] = i
    valid_df.loc[valid_df.genre==genre, 'class_genre'] = i
    class_dict_genre.update({i:genre})
pd.concat([train_df.genre.value_counts().to_frame('train_counts'), valid_df.genre.value_counts().to_frame('valid_counts')],1)

In [ ]:
train_df = train_df[train_df.genre.isin(['pastorale','miniature','bird-and-flower painting','yakusha-e',
                              'advertisement','calligraphy','bijinga','vanitas','shan shui','quadratura','panorama','tessellation','cloudscape'])==False]
class_dict_genre = {}
for i, genre in enumerate(np.sort(train_df.genre.unique())):
    train_df.loc[train_df.genre==genre, 'class_genre'] = i
    class_dict_genre.update({i:genre})

class_dict_genre

#### Clip model

In [ ]:
from clip.model import build_model
import clip

In [ ]:
clip_model, preprocess = clip.load("ViT-B/32", device)
clip_model.float()

In [ ]:
train_df = train_df.reset_index(drop=True)

In [ ]:
tmp_train = train_df.groupby('genre', group_keys=False).apply(lambda df: df.sample(142))

In [ ]:
features, labels = [], []
for i, row in tqdm(tmp_train.iterrows()):
    image = Image.open(row.img_path).convert('RGB')
    img_tr = preprocess(image).unsqueeze(0)
    out = clip_model.visual(img_tr.to(device))
    features.append(out.detach().cpu().numpy())
    labels.append(row.class_genre)

In [ ]:
feat = np.concatenate(features)

In [ ]:
feat.shape

In [ ]:
out = pd.DataFrame(feat)
out['labels'] = labels
out = out.groupby('labels').mean()

In [ ]:
clustering = AgglomerativeClustering(n_clusters=17).fit(out.values)

In [ ]:
clusters = pd.DataFrame([class_dict_genre]).T
clusters = clusters[0].to_frame('genre')
clusters['cluster'] = clustering.labels_
clusters = clusters.sort_values('cluster')
clusters = clusters.merge(train_df.groupby('genre').size().to_frame('counts').reset_index())

In [ ]:
clusters

In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color=(256, 256, 256))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (max(im1.width,im2.width), im1.height + im2.height), color=(256, 256, 256))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def images_block(df, shape=(224,224)):
    im = image_processing(Image.open(df.iloc[0].img_path).convert('RGB'), pad_color='white', shape=shape)
    for i in range(1,len(df)):
        im = get_concat_h(im,image_processing(Image.open(df.iloc[i].img_path).convert('RGB'), pad_color='white', shape=shape))
    return im

def image_processing(x, shape=306, pad_color='black'):
    desired_size = max(x.size)
    new_size = x.size
    delta_w = desired_size - new_size[0]
    delta_h = desired_size - new_size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    if pad_color == 'white':
        new_im = ImageOps.expand(x, padding, fill=(256, 256, 256))
    else:
        new_im = ImageOps.expand(x, padding)
    new_im = transforms.Compose([transforms.Resize(shape)])(new_im)
    return new_im

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==11].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==1].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==2].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==3].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==4].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==5].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==6].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==8].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
tmp = train_df[train_df.genre.isin(clusters[clusters.cluster==9].genre)]
print(tmp.groupby('genre').size().values)
tmp = tmp.groupby('genre', group_keys=False).apply(lambda df: df.sample(1))
print(tmp.genre.tolist())
images_block(tmp)

In [ ]:
X = np.asarray(feat, dtype='float32')
X_embedded = TSNE(n_components=2, init='random').fit_transform(X)

In [ ]:
df = pd.DataFrame(X_embedded)
df['labels'] = np.concatenate(np.array(labels).reshape(-1,1))
# k = random.choices(range(41), k=15)
# df = df[df.labels.isin(k)]
df['labels'] = df.labels.apply(lambda x: class_dict_genre[x])

In [ ]:
means = df.groupby('labels').mean().reset_index()

In [ ]:
from scipy.spatial import distance
a = (1, 2, 3)
b = (4, 5, 6)
dst = distance.euclidean(a, b)

In [ ]:
dst

In [ ]:
tuple(means.loc[means.labels=='abstract'][[0,1]].values[0])

In [ ]:
means

In [ ]:
means['wildlife_painting_dst'] = means.apply(lambda x: distance.euclidean((x[0],x[1]),tuple(means.loc[means.labels=='wildlife painting'][[0,1]].values[0])), 1)

In [ ]:
colors = ["#FF5733","#DAF7A6","#C0C0C0","#FFC300","#C70039",
          "#FF00FF","#008000","#3498DB","#2471A3","#7D3C98",
          "#00FFFF","#7D6608","#00FF00","#000000"]
pal = sns.color_palette(colors)
tmp = means.sort_values('wildlife_painting_dst').head(14) #[(means[0]<0)&(means[1]<0)]

fig = plt.figure(figsize=(15, 11))
sns.scatterplot(data=tmp, x=tmp[0], y=tmp[1], hue="labels", palette=pal, s=300, alpha=1)

In [ ]:
tmp

In [ ]:
df

In [ ]:
fig = plt.figure(figsize=(15, 11))
sns.kdeplot(
    data=df[df.labels.isin(['still life', 'vanitas'])],
    x=df.columns[0],
    y=df.columns[1],
    hue="labels",
    thresh=.1,
    palette='turbo',
    fill=True,
    alpha=.5
)

In [ ]:
tmp = df[df.labels.isin(['still life', 'vanitas'])]
fig = plt.figure(figsize=(15, 11))
sns.scatterplot(data=tmp, x=tmp[0], y=tmp[1], hue="labels", palette='turbo', s=300, alpha=1)